## Dataframe II

In [19]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local") \
    .appName("Nome do Aplicativo") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

23/06/10 13:32:23 WARN Utils: Your hostname, big-linux resolves to a loopback address: 127.0.1.1; using 192.168.0.111 instead (on interface wlp2s0)
23/06/10 13:32:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/10 13:32:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
df1 = spark.createDataFrame([('Pedro', 10),('Maria', 20),('José', 40)])
df1.show()

+-----+---+
|   _1| _2|
+-----+---+
|Pedro| 10|
|Maria| 20|
| José| 40|
+-----+---+



In [7]:
schema = 'Id INT,Nome STRING'
dados = [[1,'Pedro'],[2,'Maria']]
df2 = spark.createDataFrame(dados, schema)
df2.show()

+---+-----+
| Id| Nome|
+---+-----+
|  1|Pedro|
|  2|Maria|
+---+-----+



In [8]:
df2.show(1)

+---+-----+
| Id| Nome|
+---+-----+
|  1|Pedro|
+---+-----+
only showing top 1 row



In [9]:
from pyspark.sql.functions import sum
schema2 = 'Produtos String, Vendas INT'
vendas = [['Caneta', 10], ['Lápis', 20 ], ['Caneta', 40]]
df3 = spark.createDataFrame(vendas, schema2)
df3.show()

+--------+------+
|Produtos|Vendas|
+--------+------+
|  Caneta|    10|
|   Lápis|    20|
|  Caneta|    40|
+--------+------+



In [10]:
agrupado = df3.groupBy('Produtos').agg(sum('Vendas'))
agrupado.show()

+--------+-----------+
|Produtos|sum(Vendas)|
+--------+-----------+
|  Caneta|         50|
|   Lápis|         20|
+--------+-----------+



In [11]:
agrupado = df3.groupBy('Produtos').agg(sum('Vendas')).show()

+--------+-----------+
|Produtos|sum(Vendas)|
+--------+-----------+
|  Caneta|         50|
|   Lápis|         20|
+--------+-----------+



In [12]:
df3.select('Produtos').show()

+--------+
|Produtos|
+--------+
|  Caneta|
|   Lápis|
|  Caneta|
+--------+



In [13]:
df3.select('Produtos', 'Vendas').show()

+--------+------+
|Produtos|Vendas|
+--------+------+
|  Caneta|    10|
|   Lápis|    20|
|  Caneta|    40|
+--------+------+



In [14]:
df3.select('Vendas', 'Produtos').show()

+------+--------+
|Vendas|Produtos|
+------+--------+
|    10|  Caneta|
|    20|   Lápis|
|    40|  Caneta|
+------+--------+



In [15]:
from pyspark.sql.functions import expr # expressão
df3.select('Produtos', 'Vendas', expr('Vendas * 0.2')).show()

+--------+------+--------------+
|Produtos|Vendas|(Vendas * 0.2)|
+--------+------+--------------+
|  Caneta|    10|           2.0|
|   Lápis|    20|           4.0|
|  Caneta|    40|           8.0|
+--------+------+--------------+



## Dataframe III

In [16]:
df3.schema

StructType([StructField('Produtos', StringType(), True), StructField('Vendas', IntegerType(), True)])

In [17]:
df3.columns

['Produtos', 'Vendas']

In [41]:
# Ingestão de dados
arqschema = 'Id INT, nome STRING, status STRING, cidade STRING, \
             vendas INT, data STRING'

In [42]:
despachantes = spark.read.csv('/home/heliton/Documentos/repos/\
novo_aws_cloud_data_engineer_compass_uol/\
sprint_7/curso_spark/download/despachantes.csv',\
header=False, schema=arqschema)

In [25]:
despachantes.show()

+---+-------------------+------+-------------+------+----------+
| Id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [39]:
desp_autoschema = spark.read.load('/home/heliton/Documentos/repos/\
novo_aws_cloud_data_engineer_compass_uol/\
sprint_7/curso_spark/download/despachantes.csv',\
header=False, format='csv', sep=',', inferSchema=True)

In [35]:
desp_autoschema.show()

+---+-------------------+-----+-------------+---+----------+
|_c0|                _c1|  _c2|          _c3|_c4|       _c5|
+---+-------------------+-----+-------------+---+----------+
|  1|   Carminda Pestana|Ativo|  Santa Maria| 23|2020-08-11|
|  2|    Deolinda Vilela|Ativo|Novo Hamburgo| 34|2020-03-05|
|  3|   Emídio Dornelles|Ativo| Porto Alegre| 34|2020-02-05|
|  4|Felisbela Dornelles|Ativo| Porto Alegre| 36|2020-02-05|
|  5|     Graça Ornellas|Ativo| Porto Alegre| 12|2020-02-05|
|  6|   Matilde Rebouças|Ativo| Porto Alegre| 22|2019-01-05|
|  7|    Noêmia   Orriça|Ativo|  Santa Maria| 45|2019-10-05|
|  8|      Roque Vásquez|Ativo| Porto Alegre| 65|2020-03-05|
|  9|      Uriel Queiroz|Ativo| Porto Alegre| 54|2018-05-05|
| 10|   Viviana Sequeira|Ativo| Porto Alegre|  0|2020-09-05|
+---+-------------------+-----+-------------+---+----------+



In [40]:
despachantes.schema

StructType([StructField('Id', IntegerType(), True), StructField('nome', StringType(), True), StructField('status', StringType(), True), StructField('cidade', StringType(), True), StructField('vendas', IntegerType(), True), StructField('data', StringType(), True)])

In [41]:
desp_autoschema.schema

StructType([StructField('_c0', IntegerType(), True), StructField('_c1', StringType(), True), StructField('_c2', StringType(), True), StructField('_c3', StringType(), True), StructField('_c4', IntegerType(), True), StructField('_c5', TimestampType(), True)])

## Dataframe IV

In [43]:
from pyspark.sql import functions

In [45]:
despachantes.select('id','nome','vendas')\
    .where(functions.col('vendas') > 20).show()

+---+-------------------+------+
| id|               nome|vendas|
+---+-------------------+------+
|  1|   Carminda Pestana|    23|
|  2|    Deolinda Vilela|    34|
|  3|   Emídio Dornelles|    34|
|  4|Felisbela Dornelles|    36|
|  6|   Matilde Rebouças|    22|
|  7|    Noêmia   Orriça|    45|
|  8|      Roque Vásquez|    65|
|  9|      Uriel Queiroz|    54|
+---+-------------------+------+



In [53]:
despachantes.select('id','nome','vendas').where((functions.col('vendas') > 20) & (functions.col('vendas') < 40)).show()

+---+-------------------+------+
| id|               nome|vendas|
+---+-------------------+------+
|  1|   Carminda Pestana|    23|
|  2|    Deolinda Vilela|    34|
|  3|   Emídio Dornelles|    34|
|  4|Felisbela Dornelles|    36|
|  6|   Matilde Rebouças|    22|
+---+-------------------+------+



In [54]:
novodf = despachantes.withColumnRenamed('nome', 'nomes')

In [55]:
novodf.columns

['Id', 'nomes', 'status', 'cidade', 'vendas', 'data']

In [57]:
from pyspark.sql.functions import *

In [61]:
despachantesData = despachantes.withColumn(\
    'data2', to_timestamp(functions.col('data'), 'yyyy-MM-dd'))

In [62]:
despachantesData.schema

StructType([StructField('Id', IntegerType(), True), StructField('nome', StringType(), True), StructField('status', StringType(), True), StructField('cidade', StringType(), True), StructField('vendas', IntegerType(), True), StructField('data', StringType(), True), StructField('data2', TimestampType(), True)])

In [64]:
despachantesData.select(year('data')).show()

+----------+
|year(data)|
+----------+
|      2020|
|      2020|
|      2020|
|      2020|
|      2020|
|      2019|
|      2019|
|      2020|
|      2018|
|      2020|
+----------+



In [69]:
despachantesData.select(year('data')).distinct().show()

+----------+
|year(data)|
+----------+
|      2018|
|      2019|
|      2020|
+----------+



In [70]:
despachantesData.select('nome',year('data')).orderBy('nome').show()

+-------------------+----------+
|               nome|year(data)|
+-------------------+----------+
|   Carminda Pestana|      2020|
|    Deolinda Vilela|      2020|
|   Emídio Dornelles|      2020|
|Felisbela Dornelles|      2020|
|     Graça Ornellas|      2020|
|   Matilde Rebouças|      2019|
|    Noêmia   Orriça|      2019|
|      Roque Vásquez|      2020|
|      Uriel Queiroz|      2018|
|   Viviana Sequeira|      2020|
+-------------------+----------+



In [74]:
# Contar as datas
despachantesData.select('data').groupBy(year('data')).count().show()

+----------+-----+
|year(data)|count|
+----------+-----+
|      2018|    1|
|      2019|    2|
|      2020|    7|
+----------+-----+



In [75]:
despachantesData.select(sum('vendas')).show()

+-----------+
|sum(vendas)|
+-----------+
|        325|
+-----------+



## Ações e transformações

In [76]:
despachantes.show()

+---+-------------------+------+-------------+------+----------+
| Id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [77]:
despachantes.take(1)

[Row(Id=1, nome='Carminda Pestana', status='Ativo', cidade='Santa Maria', vendas=23, data='2020-08-11')]

In [78]:
despachantes.head(1)

[Row(Id=1, nome='Carminda Pestana', status='Ativo', cidade='Santa Maria', vendas=23, data='2020-08-11')]

In [79]:
despachantes.collect()

[Row(Id=1, nome='Carminda Pestana', status='Ativo', cidade='Santa Maria', vendas=23, data='2020-08-11'),
 Row(Id=2, nome='Deolinda Vilela', status='Ativo', cidade='Novo Hamburgo', vendas=34, data='2020-03-05'),
 Row(Id=3, nome='Emídio Dornelles', status='Ativo', cidade='Porto Alegre', vendas=34, data='2020-02-05'),
 Row(Id=4, nome='Felisbela Dornelles', status='Ativo', cidade='Porto Alegre', vendas=36, data='2020-02-05'),
 Row(Id=5, nome='Graça Ornellas', status='Ativo', cidade='Porto Alegre', vendas=12, data='2020-02-05'),
 Row(Id=6, nome='Matilde Rebouças', status='Ativo', cidade='Porto Alegre', vendas=22, data='2019-01-05'),
 Row(Id=7, nome='Noêmia   Orriça', status='Ativo', cidade='Santa Maria', vendas=45, data='2019-10-05'),
 Row(Id=8, nome='Roque Vásquez', status='Ativo', cidade='Porto Alegre', vendas=65, data='2020-03-05'),
 Row(Id=9, nome='Uriel Queiroz', status='Ativo', cidade='Porto Alegre', vendas=54, data='2018-05-05'),
 Row(Id=10, nome='Viviana Sequeira', status='Ativo', c

In [81]:
f'numero de linhas é {despachantes.count()}'

'numero de linhas é 10'

In [82]:
despachantes.show()

+---+-------------------+------+-------------+------+----------+
| Id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [83]:
# Ordenar
despachantes.select('*').orderBy('vendas').show()

+---+-------------------+------+-------------+------+----------+
| Id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
+---+-------------------+------+-------------+------+----------+



In [60]:
# Ordenar
despachantes.select('*').orderBy(col('vendas').desc()).show()

+---+-------------------+------+-------------+------+----------+
| Id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [93]:
despachantes.select('*').orderBy(col('cidade').desc(), col('vendas').desc()).show()

+---+-------------------+------+-------------+------+----------+
| Id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
+---+-------------------+------+-------------+------+----------+



In [98]:
# Agrupar
despachantes.select('*').groupBy('cidade').agg(sum('vendas')).show()

+-------------+-----------+
|       cidade|sum(vendas)|
+-------------+-----------+
|  Santa Maria|         68|
|Novo Hamburgo|         34|
| Porto Alegre|        223|
+-------------+-----------+



In [101]:
despachantes.groupBy('cidade').agg(sum('vendas')).orderBy(col('sum(vendas)').desc()).show()

+-------------+-----------+
|       cidade|sum(vendas)|
+-------------+-----------+
| Porto Alegre|        223|
|  Santa Maria|         68|
|Novo Hamburgo|         34|
+-------------+-----------+



In [46]:
despachantes.show()

+---+-------------------+------+-------------+------+----------+
| Id|               nome|status|       cidade|vendas|      data|
+---+-------------------+------+-------------+------+----------+
|  1|   Carminda Pestana| Ativo|  Santa Maria|    23|2020-08-11|
|  2|    Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
|  3|   Emídio Dornelles| Ativo| Porto Alegre|    34|2020-02-05|
|  4|Felisbela Dornelles| Ativo| Porto Alegre|    36|2020-02-05|
|  5|     Graça Ornellas| Ativo| Porto Alegre|    12|2020-02-05|
|  6|   Matilde Rebouças| Ativo| Porto Alegre|    22|2019-01-05|
|  7|    Noêmia   Orriça| Ativo|  Santa Maria|    45|2019-10-05|
|  8|      Roque Vásquez| Ativo| Porto Alegre|    65|2020-03-05|
|  9|      Uriel Queiroz| Ativo| Porto Alegre|    54|2018-05-05|
| 10|   Viviana Sequeira| Ativo| Porto Alegre|     0|2020-09-05|
+---+-------------------+------+-------------+------+----------+



In [57]:
# Filtro
despachantes.filter(col('nome')=='Deolinda Vilela').show()

+---+---------------+------+-------------+------+----------+
| Id|           nome|status|       cidade|vendas|      data|
+---+---------------+------+-------------+------+----------+
|  2|Deolinda Vilela| Ativo|Novo Hamburgo|    34|2020-03-05|
+---+---------------+------+-------------+------+----------+



## Exportando dados (salvando)

In [104]:
despachantes.write.format('parquet').save('/home/heliton/Documentos/repos/novo_aws_cloud_data_engineer_compass_uol/sprint_7/curso_spark/df_import_parquet'
                                          )

In [105]:
despachantes.write.format('csv').save('/home/heliton/Documentos/repos/novo_aws_cloud_data_engineer_compass_uol/sprint_7/curso_spark/df_import_csv'
                                          )

In [106]:
despachantes.write.format('json').save('/home/heliton/Documentos/repos/novo_aws_cloud_data_engineer_compass_uol/sprint_7/curso_spark/df_import_json'
                                          )

In [107]:
despachantes.write.format('orc').save('/home/heliton/Documentos/repos/novo_aws_cloud_data_engineer_compass_uol/sprint_7/curso_spark/df_import_orc'
                                          )

## Importando dados

In [1]:
!pwd

/home/heliton/Documentos/repos/novo_aws_cloud_data_engineer_compass_uol/sprint_7/curso_spark


In [3]:
!ls -l

total 248
-rw-r--r-- 1 heliton heliton  33088 jun 10 13:16 dataframe2.ipynb
-rw-r--r-- 1 heliton heliton    256 jun  9 14:19 dataframe3.ipynb
drwxr-xr-x 1 heliton heliton    276 jun  9 15:30 df_import_csv
drwxr-xr-x 1 heliton heliton    280 jun  9 15:31 df_import_json
drwxr-xr-x 1 heliton heliton    304 jun  9 15:31 df_import_orc
drwxr-xr-x 1 heliton heliton    320 jun  9 15:29 df_import_parquet
-rw-r--r-- 1 heliton heliton    463 jun  6 15:11 Dockerfile
drwxr-xr-x 1 heliton heliton    172 jun  9 14:25 download
-rw-r--r-- 1 heliton heliton 207452 nov 10  2021 download.zip


In [11]:
!cd df_import_csv && ls -l && mv part-00000-75aa85c8-aa25-4ad5-a330-410210a35dff-c000.csv despachantes.csv && ls -l

total 4
-rw-r--r-- 1 heliton heliton 518 jun  9 15:30 despachantes.csv
-rw-r--r-- 1 heliton heliton   0 jun  9 15:30 _SUCCESS
mv: cannot stat 'part-00000-75aa85c8-aa25-4ad5-a330-410210a35dff-c000.csv': No such file or directory


In [12]:
!cd df_import_json && ls -l && mv part-00000-8091cb4f-3691-42db-a9a7-c7e148ee4d17-c000.json despachantes.json && ls -l

total 4
-rw-r--r-- 1 heliton heliton 1078 jun  9 15:31 part-00000-8091cb4f-3691-42db-a9a7-c7e148ee4d17-c000.json
-rw-r--r-- 1 heliton heliton    0 jun  9 15:31 _SUCCESS
total 4
-rw-r--r-- 1 heliton heliton 1078 jun  9 15:31 despachantes.json
-rw-r--r-- 1 heliton heliton    0 jun  9 15:31 _SUCCESS


In [15]:
!cd df_import_orc && ls -l && mv part-00000-c04adaf4-3ff5-4906-8aa6-6fa3af12afb5-c000.snappy.orc despachantes.orc && ls -l

total 4
-rw-r--r-- 1 heliton heliton 1325 jun  9 15:31 despachantes.orc
-rw-r--r-- 1 heliton heliton    0 jun  9 15:31 _SUCCESS
mv: cannot stat 'part-00000-c04adaf4-3ff5-4906-8aa6-6fa3af12afb5-c000.snappy.orc': No such file or directory


In [17]:
!cd df_import_parquet && ls -l && mv part-00000-7c53d34b-445b-4b69-ba09-bf55f025985a-c000.snappy.parquet despachantes.parquet && ls -l

total 4
-rw-r--r-- 1 heliton heliton 2167 jun  9 15:29 despachantes.parquet
-rw-r--r-- 1 heliton heliton    0 jun  9 15:29 _SUCCESS
mv: cannot stat 'part-00000-7c53d34b-445b-4b69-ba09-bf55f025985a-c000.snappy.parquet': No such file or directory


In [26]:
parquet = spark.read.format('parquet').load('/home/heliton/Documentos/repos/novo_aws_cloud_data_engineer_compass_uol/sprint_7/curso_spark/df_import_parquet/despachantes.parquet')

In [27]:
parquet.schema

StructType([StructField('Id', IntegerType(), True), StructField('nome', StringType(), True), StructField('status', StringType(), True), StructField('cidade', StringType(), True), StructField('vendas', IntegerType(), True), StructField('data', StringType(), True)])

In [28]:
json = spark.read.format('json').load('/home/heliton/Documentos/repos/novo_aws_cloud_data_engineer_compass_uol/sprint_7/curso_spark/df_import_json/despachantes.json')

In [29]:
json.schema

StructType([StructField('Id', LongType(), True), StructField('cidade', StringType(), True), StructField('data', StringType(), True), StructField('nome', StringType(), True), StructField('status', StringType(), True), StructField('vendas', LongType(), True)])

In [24]:
orc = spark.read.format('orc').load('/home/heliton/Documentos/repos/novo_aws_cloud_data_engineer_compass_uol/sprint_7/curso_spark/df_import_orc/despachantes.orc')

In [25]:
orc.schema

StructType([StructField('Id', IntegerType(), True), StructField('nome', StringType(), True), StructField('status', StringType(), True), StructField('cidade', StringType(), True), StructField('vendas', IntegerType(), True), StructField('data', StringType(), True)])

In [30]:
csv = spark.read.format('csv').load('/home/heliton/Documentos/repos/novo_aws_cloud_data_engineer_compass_uol/sprint_7/curso_spark/df_import_csv/despachantes.csv')

In [31]:
csv.schema

StructType([StructField('_c0', StringType(), True), StructField('_c1', StringType(), True), StructField('_c2', StringType(), True), StructField('_c3', StringType(), True), StructField('_c4', StringType(), True), StructField('_c5', StringType(), True)])

In [38]:
csv2 = spark.read.format('csv').load('/home/heliton/Documentos/repos/novo_aws_cloud_data_engineer_compass_uol/sprint_7/curso_spark/df_import_csv/despachantes.csv', schema=arqschema)

In [39]:
csv2.schema

StructType([StructField('Id', IntegerType(), True), StructField('nome', StringType(), True), StructField('status', StringType(), True), StructField('cidade', StringType(), True), StructField('vendas', IntegerType(), True), StructField('data', StringType(), True)])